## 1. Ophalen van de Manifesto Project-database

Dit notebook haalt de eerst algemene, geaggregeerde dataset van het Manifesto Project op, en daarna de coderingen van vijftien Westerse landen op zinsniveau. Voor dat laatste is een API key vereist. Die verkrijg je door een [account aan te maken](https://manifesto-project.wzb.eu/signup) op de website van het Manifesto Project.

In [1]:
import wget

wget.download('https://manifesto-project.wzb.eu/down/data/2023a/datasets/MPDataset_MPDS2023a.csv', 'data')

 32% [..................                                        ]  999424 / 3110688

100% [..........................................................] 3110688 / 3110688

'data/MPDataset_MPDS2023a (1).csv'

In [2]:
API_KEY = ''

In [3]:
import datetime

import requests
import pandas as pd
from tqdm.auto import tqdm

In [4]:
data = pd.read_csv('data/MPDataset_MPDS2023a.csv', low_memory=False)
data.head(5)

,country,countryname,oecdmember,eumember,edate,date,party,partyname,partyabbrev,parfam,...,per608_3,per703_1,per703_2,rile,planeco,markeco,welfare,intpeace,datasetversion,id_perm
0,11,Sweden,0,0.0,17/09/1944,194409,11220,Communist Party of Sweden,SKP,20,...,NaN,NaN,NaN,9.60,1.9,1.900,0.0,1.900,2023a,JN1LZH
1,11,Sweden,0,0.0,17/09/1944,194409,11320,Social Democratic Labour Party,SAP,30,...,NaN,NaN,NaN,-37.80,3.3,2.200,33.4,5.600,2023a,CMR7F6
2,11,Sweden,0,0.0,17/09/1944,194409,11420,People’s Party,FP,40,...,NaN,NaN,NaN,9.50,3.2,6.400,14.3,1.600,2023a,Z6OL6C
3,11,Sweden,0,0.0,17/09/1944,194409,11620,Right Party,NaN,60,...,NaN,NaN,NaN,28.00,1.8,22.800,10.6,0.000,2023a,YMKVN2
4,11,Sweden,0,0.0,17/09/1944,194409,11810,Agrarian Party,NaN,80,...,NaN,NaN,NaN,23.81,0.0,19.048,0.0,4.762,2023a,U4SCRD


In [5]:
SELECTED_COUNTRIES = ['United States', 'Canada', 'Spain', 'Belgium', 'Netherlands', 'France', 'Denmark',
 'Italy', 'Austria', 'Germany', 'Switzerland', 'Sweden', 'United Kingdom', 'Portugal', 'Finland']

data = data[data.countryname.isin(SELECTED_COUNTRIES)]
data['date'] = pd.to_datetime(data.edate, dayfirst=True)
data = data[data.date > datetime.datetime(2000,1,1)]

In [6]:
def manifesto_sentences(manifesto_id):
    url = "https://manifesto-project.wzb.eu/api/v1/texts_and_annotations?api_key={}&keys[]={}&version=2023-1"
    url = url.format(API_KEY, manifesto_id)
    response = requests.get(url).json()
    return response

rows = []
for row in tqdm(data.itertuples(), total=len(data)):
    manifesto_id = str(row.party) + '_' + str(row.date)[:7].replace('-', '')
    response = manifesto_sentences(manifesto_id)
    if 'items' in response:
        for item in response['items']:
            for i in item['items']:
                i['party'] = row.partyabbrev
                i['date'] = row.date
                i['country'] = row.countryname
                rows.append(i)

  0%|          | 0/706 [00:00<?, ?it/s]

In [7]:
manifestos = pd.DataFrame(rows)

selected_cols = ['101', '102', '103', '104', '105', '106', '107', '108', '109',
                 '110', '201', '202', '203', '204', '301', '302', '303', '304',
                 '305', '401', '402', '403', '404', '405', '406', '407', '408',
                 '409', '410', '411', '412', '413', '414', '415', '416', '501', 
                 '502', '503', '504', '505', '506', '507', '601', '602', '603',
                 '604', '605', '606', '607', '608', '701', '702', '703', '704',
                 '705', '706']

manifestos.loc[manifestos['party'] == 'D’66', 'party'] = 'D66'
manifestos['code'] = manifestos.cmp_code.apply(lambda x: x[:3] if str(x[:3]) in selected_cols else '000')

In [8]:
manifestos[['text', 'party', 'country', 'date', 'code'
           ]].to_csv('data/manifestos_2000_2022_coded_sents.csv', index=False)